In [1]:
from stellargraph import StellarGraph
import stellargraph as sg
from stellargraph.mapper import PaddedGraphGenerator
from stellargraph.layer import DeepGraphCNN
from IPython.display import display, HTML
import pandas as pd
import numpy as np
import json
import glob
import os
import sys

2023-01-18 12:16:43.139259: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-18 12:16:43.338392: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-18 12:16:43.338409: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-18 12:16:44.087969: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
def read_files(prop, value):
    malware = {}
    exe_files = []
    for i in glob.glob('/home/poonia/Documents/opcode_dataset/dataset/PE/'+prop+'/*.exe'):
        if(prop == 'malicious'):
            exe_files.append(i.split('/')[-1].replace('.exe', '_malicious.exe'))
            f_hash = i.split('/')[-1].split('.')[0]
            malware[f_hash] = {}
            malware[f_hash]['value'] = value
        else:
            exe_files.append(i.split('/')[-1])
            f_hash = i.split('/')[-1].split('_')[0]
            malware[f_hash] = {}
            malware[f_hash]['value'] = value
    edges_files = []
    for i in glob.glob('/home/poonia/Documents/opcode_dataset/dataset/PE/'+prop+'/*.edge'):
        edges_files.append(i.split('/')[-1])
        f_hash = i.split('/')[-1].split('_')[0]
        malware[f_hash]['edges'] = i.split('/')[-1]
    node_files = []
    for i in glob.glob('/home/poonia/Documents/opcode_dataset/dataset/PE/'+prop+'/*.json'):
        node_files.append(i.split('/')[-1])
        f_hash = i.split('/')[-1].split('_')[0]
        malware[f_hash]['nodes']=i.split('/')[-1]
    opcode_seq_files = []
    for i in glob.glob('/home/poonia/Documents/opcode_dataset/dataset/PE/'+prop+'/*.opcode_seq'):
        opcode_seq_files.append(i.split('/')[-1])
        f_hash = i.split('/')[-1].split('_')[0]
        malware[f_hash]['opcode_seq'] = i.split('/')[-1]
    return malware

In [3]:
def node_conversion(node_file):
    data = pd.read_json(node_file)
    np_data = np.array(data)
    np_data_transpose = np_data.transpose()
    df = pd.DataFrame(np_data_transpose, columns=['dest', 'src'])
    one_hot_encoding = pd.get_dummies(df, columns=['dest', 'src'])
    return one_hot_encoding

In [4]:
def load(filename, prop):
    edges_file = 'dataset/PE/'+prop+'/'+filename['edges'].strip()
    opcode_seq = 'dataset/PE/'+prop+'/'+filename['opcode_seq'].strip()
    nodes_file = 'dataset/PE/'+prop+'/'+filename['nodes'].strip()
    edges_from = []
    edges_to = []
    weights = []

    with open(edges_file, 'r') as f:
        edges = f.readlines()
        for edge in edges:
            tmp = edge.strip().split()
            t = tmp[0].strip().split('->')
            edges_from.append(t[0])
            edges_to.append(t[1])
            weights.append(tmp[1])
    g_edges = pd.DataFrame({"source":edges_from, "target":edges_to, "weights":weights})
    
    with open('node_feature_mapping.json', 'r') as f:
        operands = json.load(f)
    
    with open(nodes_file, 'r') as f:
        node_feature = json.load(f)
    #Nodes
    index = []
    dest = []
    src = []
    for node in node_feature:
        index.append(node)
        if(operands[node_feature[node]['dest']] == 0):
            dest.append(0)
        else:
            dest.append(1)
        if(operands[node_feature[node]['src']] == 0):
            src.append(0)
        else:
            src.append(1)

    g_nodes = pd.DataFrame({"dest":dest, "src":src}, index=index)
    #g_nodes = pd.DataFrame(node_conversion(nodes_file), index=index)
    #print(g_nodes)
    G = StellarGraph(g_nodes, g_edges)
    return G
    

In [5]:
def dataset(files, hash_to_num):
    graph = {}
    prop = ''
    for i in files:
        if(files[i]['value'] == 1):
            prop = 'malicious'
        else:
            prop = 'benign'
        g = load(files[i], prop)
        graph[hash_to_num[str(i)]] = []
        graph[hash_to_num[str(i)]].append(g)
     
    return graph
        
        

In [23]:
import itertools
import random

malware = read_files('malicious', 1)
benign = read_files('benign', 0)
dic = {}
#dic.update(dict(itertools.islice(malware.items(), 10000)))
#dic.update(benign)
dic.update(dict(itertools.islice(malware.items(), 1500)))
dic.update(dict(itertools.islice(benign.items(), 1500)))
l = list(dic.items())
random.shuffle(l)
dic = dict(l)
print(len(dic))
d = {}
hash_to_num = {}
count = 0
for i in dic:
    if(dic[i]['value'] == 1):
        d[str(count)] = 1
        hash_to_num[str(i)] = str(count)
    else:
        d[str(count)] = 0
        hash_to_num[str(i)] = str(count)
    count+=1
        

df = pd.DataFrame([d])
a = np.array(df)
t = a.transpose()
f_df = pd.DataFrame(t, columns=['Labels'])
labels = pd.get_dummies(f_df, columns=['Labels'])
labels.head()
    
graph_ref = dataset(dic, hash_to_num)



3000


In [24]:
#d = pd.DataFrame(graph_ref, columns=['Graph', 'Labels'])
from sklearn import model_selection

graph_labels = f_df
train_graphs, test_graphs = model_selection.train_test_split(
    graph_labels, train_size=0.9, test_size=None, stratify=graph_labels,
)
print(test_graphs)

graphs = []
for i in graph_ref:
    graphs.append(graph_ref[i][0])
generator = PaddedGraphGenerator(graphs=graphs)
#graph_list = list(graph_ref.values())
#graph_list

      Labels
1878       0
2569       0
672        1
1893       0
2544       0
...      ...
2864       1
877        1
1447       1
80         1
1741       1

[300 rows x 1 columns]


In [25]:
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Conv1D, MaxPool1D, Dropout, Flatten
from tensorflow.keras.losses import binary_crossentropy
import tensorflow as tf



In [26]:
print(generator.node_features_size)
k = 35  # the number of rows for the output tensor
layer_sizes = [32, 32, 32, 1]

dgcnn_model = DeepGraphCNN(
    layer_sizes=layer_sizes,
    activations=["tanh", "tanh", "tanh", "tanh"],
    generator=generator,
    k=k
)
x_inp, x_out = dgcnn_model.in_out_tensors()


3


In [27]:
x_out = Conv1D(filters=16, kernel_size=sum(layer_sizes), strides=sum(layer_sizes))(x_out)
x_out = MaxPool1D(pool_size=2)(x_out)

x_out = Conv1D(filters=32, kernel_size=5, strides=1)(x_out)

x_out = Flatten()(x_out)

x_out = Dense(units=128, activation="relu")(x_out)
x_out = Dropout(rate=0.5)(x_out)

predictions = Dense(units=1, activation="sigmoid")(x_out)

In [28]:
model = Model(inputs=x_inp, outputs=predictions)

model.compile(
    optimizer=Adam(learning_rate=0.0001), loss=binary_crossentropy, metrics=["acc"],
)

In [29]:
graph_labels = f_df
train_graphs, test_graphs = model_selection.train_test_split(
    graph_labels, train_size=0.9, test_size=None, stratify=graph_labels,
)


In [30]:
gen = PaddedGraphGenerator(graphs=graphs)
train_gen = gen.flow(
    list(train_graphs.index - 1),
    targets=train_graphs.values,
    batch_size=50,
    symmetric_normalization=False,
)

test_gen = gen.flow(
    list(test_graphs.index - 1),
    targets=test_graphs.values,
    batch_size=1,
    symmetric_normalization=False,
)

In [31]:
epochs = 50

In [32]:
history = model.fit(
    train_gen, epochs=epochs, verbose=1, validation_data=test_gen, shuffle=True,
)

MemoryError: Unable to allocate 40.2 GiB for an array with shape (73492, 73492) and data type float64

In [1]:
sg.utils.plot_history(history)

NameError: name 'sg' is not defined

In [ ]:
test_metrics = model.evaluate(test_gen)
print("\nTest Set Metrics:")
for name, val in zip(model.metrics_names, test_metrics):
    print("\t{}: {:0.4f}".format(name, val))
